In [25]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [26]:
# trainset = np.load('small_train_high_low.npy')
# testset = np.load('small_test_high_low.npy')
trainset = np.load('direction_train.npy')
testset = np.load('direction_test.npy')

In [27]:
np.any(np.isnan(trainset))

False

In [28]:
def dict_from_array(arr):
    d = {}
    for row in range(arr.shape[1]):
        d[str(row)] = arr[:,row]
    return d

def no_zeros(arr):
    arr[arr==0] = 1e-8
    return arr

In [29]:
#Taken from https://www.tensorflow.org/get_started/datasets_quickstart
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(buffer_size=30000).repeat(count=None).batch(batch_size)

    # Build the Iterator, and return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()


In [30]:
train_x = dict_from_array(no_zeros(trainset[:,:-1]))
train_y = trainset[:,-1]
test_x = dict_from_array(no_zeros(testset[:,:-1]))
test_y = testset[:,-1]
print(len(train_y))

28458


In [31]:
feature_columns = []
for i in range(len(train_x)):
    feature_columns.append(tf.feature_column.numeric_column(key=str(i)))

In [32]:
len(feature_columns)

164

In [33]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    hidden_units = [81, 40],
    n_classes = 2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Micah\\AppData\\Local\\Temp\\tmpwnt1p4qp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023A039483C8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
classifier.train(
    input_fn=lambda:train_input_fn(train_x,train_y,100), steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Micah\AppData\Local\Temp\tmpwnt1p4qp\model.ckpt.
INFO:tensorflow:loss = 92.422775, step = 1
INFO:tensorflow:global_step/sec: 61.1873
INFO:tensorflow:loss = 14.969423, step = 101 (1.636 sec)
INFO:tensorflow:global_step/sec: 111.419
INFO:tensorflow:loss = 7.3293724, step = 201 (0.897 sec)
INFO:tensorflow:global_step/sec: 30.5767
INFO:tensorflow:loss = 0.77275985, step = 301 (3.271 sec)
INFO:tensorflow:global_step/sec: 101.146
INFO:tensorflow:loss = 1.5305548, step = 401 (0.988 sec)
INFO:tensorflow:global_step/sec: 103.347
INFO:tensorflow:loss = 0.7906143, step = 501 (0.968 sec)
INFO:tensorflow:global_step/sec: 29.035
INFO:tensorflow:loss = 0.25921845, step = 601 (3.444 sec)
INFO:tensorflow:global_step/sec: 104.726
INFO:tensorflow:loss = 0.15281747, step = 701 (0.955 sec)
INFO:tensorflow:global_step/sec: 108.13
INFO:tensorflow:loss = 0.5124504, step = 801 (0.925 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 102.913
INFO:tensorflow:loss = 0.0026141058, step = 8101 (0.973 sec)
INFO:tensorflow:global_step/sec: 105.058
INFO:tensorflow:loss = 0.0053898254, step = 8201 (0.951 sec)
INFO:tensorflow:global_step/sec: 26.6088
INFO:tensorflow:loss = 0.0032148531, step = 8301 (3.759 sec)
INFO:tensorflow:global_step/sec: 102.508
INFO:tensorflow:loss = 0.0059385533, step = 8401 (0.975 sec)
INFO:tensorflow:global_step/sec: 102.593
INFO:tensorflow:loss = 0.0037871727, step = 8501 (0.976 sec)
INFO:tensorflow:global_step/sec: 25.8717
INFO:tensorflow:loss = 0.0068945154, step = 8601 (3.864 sec)
INFO:tensorflow:global_step/sec: 103.983
INFO:tensorflow:loss = 0.0027255723, step = 8701 (0.963 sec)
INFO:tensorflow:global_step/sec: 103.961
INFO:tensorflow:loss = 0.0035033403, step = 8801 (0.962 sec)
INFO:tensorflow:global_step/sec: 25.988
INFO:tensorflow:loss = 0.004979263, step = 8901 (3.848 sec)
INFO:tensorflow:global_step/sec: 96.6454
INFO:tensorflow:loss = 0.0032073827, step =

In [17]:
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 100))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-02-25-23:54:00
INFO:tensorflow:Restoring parameters from C:\Users\Micah\AppData\Local\Temp\tmp59w4y821\model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2018-02-25-23:54:04
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9971537, accuracy_baseline = 0.5075901, auc = 0.99812835, auc_precision_recall = 0.99907726, average_loss = 0.015762577, global_step = 10000, label/mean = 0.5075901, loss = 1.5575396, prediction/mean = 0.5046259

Test set accuracy: 0.997

